In [154]:
#import sys
#print(sys.version)

In [155]:
%pip install ultralytics

Note: you may need to restart the kernel to use updated packages.


In [156]:
import ultralytics
ultralytics.__version__

'8.3.9'

In [157]:
# Not training the model, just using pretrained model trained on COCO dataset. 
# Aquest dataset té som vehicles a part d'altres objectes.
#Fer servir aquest model per la detecció i després track i count
import cv2
import pandas as pd
from ultralytics import YOLO
from tracker import*

model=YOLO('yolov5n.pt')

PRO TIP  Replace 'model=yolov5n.pt' with new 'model=yolov5nu.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.



In [158]:
#list of COCO dataset classes
class_list = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

In [159]:
#creant una instancia de tracker que ve de from tracker import*
tracker=Tracker()
count=0

In [160]:
#llegir el video
cap=cv2.VideoCapture('output2.mp4')

In [161]:
#VARIABLES PER GUARDAR COTXES UP I DOWN
down={}
up={}

counter_down=[]
counter_up=[]

In [164]:
import cv2
import time
import numpy as np
import pandas as pd

# Inicializar el contador de FPS y otras variables
fps = 0
start_time = time.time()
count = 0

# Definir las coordenadas de los puntos para la región de interés
p1, p2, p3, p4 = (60, 600), (500, 600), (500, 880), (60, 880)
pts = np.array([p1, p2, p3, p4], np.float32)

# Definir el tamaño de la imagen de salida transformada
width, height = 230 - 27, 480 - 300
output_pts = np.array([[0, 0], [width, 0], [width, height], [0, height]], np.float32)

# Calcular la matriz de transformación de perspectiva
M = cv2.getPerspectiveTransform(pts, output_pts)

# Inicializar listas y diccionarios para el conteo de autos
down = {}
up = {}
counter_down = []
counter_up = []

while True:
    # Leer un nuevo frame de la captura de vídeo
    ret, frame = cap.read()
    if not ret:
        break
    count += 1

    # Saltar 1 de cada 4 frames para reducir carga de procesamiento
    if count % 4 != 0:
        fps += 1
        end_time = time.time()
        elapsed_time = end_time - start_time
        if elapsed_time > 0:
            current_fps = fps / elapsed_time
            cv2.putText(frame, f'FPS: {round(current_fps, 2)}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            cv2.line(frame, (0, 400), (800, 400), (0, 0, 255), 3)
        cv2.imshow("frames", frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break
        continue

    # Aplicar la transformación de perspectiva para obtener el área recortada
    warped_frame = cv2.warpPerspective(frame, M, (width, height))

    # Realizar la predicción con YOLO en el área recortada
    results = model.predict(warped_frame)

    # Obtener datos de las bounding boxes y ajustar coordenadas
    a = results[0].boxes.data
    a = a.detach().cpu().numpy()
    px = pd.DataFrame(a).astype("float")

    car_bboxes = []
    for index, row in px.iterrows():
        x1, y1, x2, y2, d = int(row[0]), int(row[1]), int(row[2]), int(row[3]), int(row[5])
        c = class_list[d]
        if 'car' in c:
            car_bboxes.append([x1, y1, x2, y2])

    # Proporcionar las coordenadas de las bounding boxes al tracker
    bbox_id = tracker.update(car_bboxes)

    # Procesar bounding boxes y actualizar contadores
    for bbox in bbox_id:
        x3, y3, x4, y4, id = bbox
        cx, cy = int((x3 + x4) // 2), int((y3 + y4) // 2)

        # Parámetros de conteo
        line_y = 400  # Posición de la línea de referencia
        offset = 10   # Margen de error para detección cruzada

        # Detectar cruce de línea para conteo de bajada
        if cy < line_y - offset:
            down[id] = cy
        if id in down and cy > line_y + offset:
            if id not in counter_down:
                counter_down.append(id)
                cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)
                cv2.putText(frame, f'ID:{id}', (cx, cy), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)

        # Detectar cruce de línea para conteo de subida
        if cy > line_y + offset:
            up[id] = cy
        if id in up and cy < line_y - offset:
            if id not in counter_up:
                counter_up.append(id)
                cv2.circle(frame, (cx, cy), 4, (0, 255, 0), -1)
                cv2.putText(frame, f'ID:{id}', (cx, cy), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)

    # Contar autos únicos que han bajado y subido
    downwards = len(set(counter_down))
    upwards = len(set(counter_up))

    # Mostrar resultados en consola
    print(f"Bajada: {downwards}, Subida: {upwards}")

    # Cálculo de FPS y visualización en el frame
    fps += 1
    end_time = time.time()
    elapsed_time = end_time - start_time
    if elapsed_time > 0:
        current_fps = fps / elapsed_time
        cv2.putText(frame, f'FPS: {round(current_fps, 2)}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    # Mostrar el frame actualizado
    cv2.imshow("frames", frame)
    cv2.imshow("ROI", warped_frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break

# Liberar la captura de vídeo y cerrar ventanas
cap.release()
cv2.destroyAllWindows()



0: 576x640 2 cars, 134.2ms
Speed: 5.0ms preprocess, 134.2ms inference, 1.0ms postprocess per image at shape (1, 3, 576, 640)
Bajada: 0, Subida: 0

0: 576x640 2 cars, 157.3ms
Speed: 9.6ms preprocess, 157.3ms inference, 2.0ms postprocess per image at shape (1, 3, 576, 640)
Bajada: 0, Subida: 0

0: 576x640 3 cars, 124.2ms
Speed: 5.0ms preprocess, 124.2ms inference, 2.0ms postprocess per image at shape (1, 3, 576, 640)
Bajada: 0, Subida: 0

0: 576x640 2 cars, 119.6ms
Speed: 6.0ms preprocess, 119.6ms inference, 1.0ms postprocess per image at shape (1, 3, 576, 640)
Bajada: 0, Subida: 0

0: 576x640 4 cars, 114.1ms
Speed: 5.0ms preprocess, 114.1ms inference, 1.0ms postprocess per image at shape (1, 3, 576, 640)
Bajada: 0, Subida: 0

0: 576x640 4 cars, 116.6ms
Speed: 6.0ms preprocess, 116.6ms inference, 2.0ms postprocess per image at shape (1, 3, 576, 640)
Bajada: 0, Subida: 0

0: 576x640 2 cars, 111.5ms
Speed: 6.0ms preprocess, 111.5ms inference, 1.0ms postprocess per image at shape (1, 3, 57